**IMPORTAR LIBRERIAS**

---

In [31]:
# Librerias de manejo de datos
import pandas as pd

In [45]:
# Importar conexión
from db_connection import get_connection

# Conectarse a la base de datos
conexion = get_connection()
cursor = conexion.cursor()


✅ Conexión exitosa a la base de datos


In [33]:
# Importar librerías para la conexión a la base de datos con SQLAlchemy
from db_connection import get_engine

engine = get_engine()

In [46]:
# 📌 Obtener todas las tablas
cursor.execute("SHOW TABLES;")
tablas = [fila[0] for fila in cursor.fetchall()]
print(f"Total de tablas encontradas: {len(tablas)}")


Total de tablas encontradas: 48


---

**FUNCIONES**

---

In [35]:
def borrar_lista_tablas(tablas_a_borrar):
    """
    Borra las tablas de la base de datos que se pasan como argumento.
    """
    for tabla in tablas_a_borrar:
        try:
            cursor.execute(f"DROP TABLE `{tabla}`;")
            tablas.remove(tabla)
            print(f"✅ Tabla eliminada: {tabla}")
        except Exception as e:
            print(f"❌ Error al eliminar {tabla}: {e}")
    
    # Confirmar cambios
    conexion.commit()

------

**FILTROS PARA SEGREGAR Y ELIMINAR LAS TABLAS QUE NO SON ÚTILES**


---

*PRIMER FILTRO: Tablas con menos de 2 registros*

In [36]:
# Escoger tablas a eliminar
tablas_a_borrar = []

for tabla in tablas:
    try:
        cursor.execute(f"SELECT COUNT(*) FROM `{tabla}`;")
        total = cursor.fetchone()[0]
        if total <= 1:
            tablas_a_borrar.append(tabla)
    except Exception as e:
        print(f"❌ Error evaluando tabla {tabla}: {e}")

print(f"\nTablas candidatas para eliminación ({len(tablas_a_borrar)}):")
print(tablas_a_borrar)



Tablas candidatas para eliminación (0):
[]


In [37]:
# ⚠️ Solo si tienes respaldo y estás en entorno seguro
# ELIMINAR TABLAS CANDIDATAS

# Desactivar restricciones
cursor.execute("SET FOREIGN_KEY_CHECKS=0;")

borrar_lista_tablas(tablas_a_borrar)

# Volver a activar restricciones
cursor.execute("SET FOREIGN_KEY_CHECKS=1;")



---

*SEGUNDO FILTRO: Tablas posiblemente innecesarias por el nombre y breve análisis*

In [38]:
# Imprimir los nombres de las tablas restantes
for tabla in tablas:
    print(tabla)


accounting_account_balances
accounting_accounts
accounting_movements
accounting_voucher_items
accounting_voucher_types
accounting_vouchers
billing_numberings
company_areas
contact_relationships
contact_statements
contacts
contract_salary_history
costs_and_expenses
costs_and_expenses_categories
coupons
dining_tables
document_items
documents
ecommerce_legal_info
ecommerce_purchase_orders
employee_contracts
employee_positions
employees
fixed_asset_depreciations
fixed_assets
fixed_assets_groups
headquarter_warehouses
headquarters
inventory_adjustments
inventory_groups
item_balance
item_categories
item_depreciations
item_kardex
item_subcategories
item_variations
items
payment_conditions
payments
payroll_deductions
payroll_details
payroll_incomes
payroll_providers
payrolls
price_lists
retention_concepts
retentions
retentions_applied
retentions_certificates
taxes
template_versions
warehouse_transfer_logs
warehouses


*ANÁLISIS Y CONCLUSIONES DE LA INFORMACIÓN OBTENIDA*

**Tablas clave para análisis de gestión empresarial (💡 Mantener)**

Estas son probablemente las más útiles para análisis de KPIs, dashboards o visualización de negocio:

*Categoría	//  Tablas clave*

📦 Inventario	= items, item_balance, item_kardex, warehouses, inventory_adjustments, item_categories, item_subcategories, item_variations

💼 Ventas / Documentos	= documents, document_items, contacts (clientes/proveedores)

🧾 Contabilidad	= accounting_vouchers, accounting_voucher_items, accounting_accounts, accounting_movements, retentions, retention_concepts, retentions_applied, accounting_account_balances

👥 Empleados / Nómina	= employees, employee_contracts, payrolls, payroll_details, payroll_incomes, payroll_deductions

🛒 Compras	= ecommerce_purchase_orders

📊 Precios	= price_lists

🏪 Organización	= headquarters, user_headquarters, roles, user_roles, company_areas

🧾 Impuestos	= taxes

🏭 Activos fijos	= fixed_assets, fixed_asset_depreciations, fixed_assets_groups


**Relevancia baja o dependiente del contexto (🤔 Revisar)**

*Estas podrían ser útiles dependiendo del foco del proyecto (clientes, pedidos, legal, e-commerce):*

contact_relationships, contact_accounts, contact_statements

coupons, coupon_groups (si hay ecommerce)

employee_positions, contract_salary_history

plan_electronic_documents, plan_system_controller

ecommerce_legal_info

costs_and_expenses, costs_and_expenses_categories

radian_documents (puede ser de facturación electrónica)

dining_tables (¿restaurante? puede ser irrelevante)

system_counters, system_restrictions

In [39]:
# Lista de tablas candidatas a eliminar
tablas_candidatas = [
    "api_access_tokens",
    "electronic_payroll_test_set",
    "migrations",
    "oauth_clients",
    "paynilo",
    "role_permissions",
    "template_versions",
    "templates",
    "user_data",
    "values_x_item",
    "warehouse_transfer_logs",
    "billing_numberings",
    "custom_fields",
    "configurations",
    "system_counters",
    "system_restrictions"
]

---

*TERCER FILTRO: tablas a eliminar según sus columnas y número de filas*

In [40]:

# Mostrar información muy básica de las tablas candidatas
for tabla in tablas_candidatas:
    try:
        df = pd.read_sql(f"SELECT * FROM `{tabla}` LIMIT 5", engine)
        print(f"\n📘 Tabla: {tabla}")
        print("Columnas:", df.columns.tolist())
        print("size: ", df.size)
    except Exception as e:
        print(f"\n❌ Error al cargar tabla {tabla}: {e}")


❌ Error al cargar tabla api_access_tokens: (mysql.connector.errors.ProgrammingError) 1146 (42S02): Table 'erp.api_access_tokens' doesn't exist
[SQL: SELECT * FROM `api_access_tokens` LIMIT 5]
(Background on this error at: https://sqlalche.me/e/20/f405)

❌ Error al cargar tabla electronic_payroll_test_set: (mysql.connector.errors.ProgrammingError) 1146 (42S02): Table 'erp.electronic_payroll_test_set' doesn't exist
[SQL: SELECT * FROM `electronic_payroll_test_set` LIMIT 5]
(Background on this error at: https://sqlalche.me/e/20/f405)

❌ Error al cargar tabla migrations: (mysql.connector.errors.ProgrammingError) 1146 (42S02): Table 'erp.migrations' doesn't exist
[SQL: SELECT * FROM `migrations` LIMIT 5]
(Background on this error at: https://sqlalche.me/e/20/f405)

❌ Error al cargar tabla oauth_clients: (mysql.connector.errors.ProgrammingError) 1146 (42S02): Table 'erp.oauth_clients' doesn't exist
[SQL: SELECT * FROM `oauth_clients` LIMIT 5]
(Background on this error at: https://sqlalche.m

**Tablas candidatas fuertes para eliminación (por contenido y nombre)**

Estas tablas tienen pocos registros, estructuras irrelevantes para KPIs o funciones accesorias que no impactan directamente en la gestión:

*Tabla	// Motivo principal*

api_access_tokens	Manejo técnico de tokens

electronic_payroll_test_set	Datos de prueba (test set)

migrations	Solo para migraciones del sistema

oauth_clients	Seguridad/autenticación interna

paynilo	Integración de pagos externa

role_permissions	Control de permisos internos

template_versions	Versiones de plantillas, sin datos

templates	Estructura visual, no analítica

user_data	Solo datos de área/cargo del usuario

custom_fields	Campos personalizados (metadatos)

system_counters	Estadísticas internas del sistema

system_restrictions	Restricciones de uso/licencia

values_x_item	Datos muy técnicos por item, posiblemente redundantes

warehouse_transfer_logs	Registro técnico, no necesariamente útil para KPIs

----

*CUARTO FILTRO: Investigar mas a fondo las tablas dudosas*

**⚠️ Tablas dudosas (verificar con .info() antes de decidir)**

Estas tienen algo más de contenido o potencial analítico dependiendo del negocio:

*billing_numberings:* Tiene 155 registros. Podría contener información útil para facturación si los números se asocian a documentos.

*configurations:* Solo 25 registros, pero hay que ver si son configuraciones claves que afectan reportes.



In [41]:
# Obtener el esquema de cada tabla para decidir si son candidatas a eliminación
tablas_dudosas = ['billing_numberings', 'configurations'] # Agregar mas tablas si es necesario

for tabla in tablas_dudosas:
    print(f"\n📋 Tabla: {tabla}")
    try:
        df = pd.read_sql(f"SELECT * FROM {tabla}", con=engine)
        print("▶️ .info():")
        print(df.info())
        print("▶️ Primeras filas:")
        display(df.head())
    except Exception as e:
        print(f"❌ Error al cargar {tabla}: {e}")



📋 Tabla: billing_numberings
▶️ .info():
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   id                      50 non-null     int64         
 1   document_type_id        50 non-null     int64         
 2   pos_numbering           50 non-null     int64         
 3   name                    50 non-null     object        
 4   prefix                  50 non-null     object        
 5   initial_number          50 non-null     int64         
 6   current_number          50 non-null     int64         
 7   final_number            22 non-null     float64       
 8   valid_since             22 non-null     object        
 9   valid_until             22 non-null     object        
 10  validity                22 non-null     float64       
 11  preferred               50 non-null     int64         
 12  electronic_

,id,document_type_id,pos_numbering,name,prefix,initial_number,current_number,final_number,valid_since,valid_until,...,tech_key,used,template,headquarter_id,warehouse_id,status,user_id,created_at,updated_at,contingency
0,1,11,0,Remisiones,,0,56,NaN,None,None,...,None,1,"{""id"": 1, ""name"": ""Carta"", ""template_type"": ""1""}",1,1,1,10,NaT,2024-05-24 09:54:00,0
1,2,12,0,Cotizaciones,,0,91,NaN,None,None,...,None,1,"{""id"": 3, ""name"": ""POS"", ""template_type"": ""1""}",1,1,1,10,NaT,2024-07-13 11:53:24,0
2,3,13,0,Recibos de Caja,,0,506,NaN,None,None,...,None,1,"{""id"": 1, ""name"": ""Carta"", ""template_type"": ""1""}",1,1,1,10,NaT,2024-07-17 10:58:41,0
3,4,14,0,Comprobantes de Pago,,0,113,NaN,None,None,...,None,1,"{""id"": 1, ""name"": ""Carta"", ""template_type"": ""1""}",1,1,1,0,NaT,2024-07-17 11:00:31,0
4,5,15,0,Órdenes de Compra,,0,23,NaN,None,None,...,None,1,"{""id"": 1, ""name"": ""Carta"", ""template_type"": ""1""}",1,1,1,0,NaT,2024-04-03 16:10:15,0



📋 Tabla: configurations
❌ Error al cargar configurations: (mysql.connector.errors.ProgrammingError) 1146 (42S02): Table 'erp.configurations' doesn't exist
[SQL: SELECT * FROM configurations]
(Background on this error at: https://sqlalche.me/e/20/f405)


---

**TABLAS FINALES A ELIMINAR DESPUÉS DE TODOS LOS FILTROS Y ANÁLISIS INDIVIDUAL**

Falta escoger y seleccionar individualmente 

In [42]:
tablas_a_eliminar = [
    "api_access_tokens",             # Manejo técnico de tokens
    "electronic_payroll_test_set",   # Datos de prueba (test set)
    "migrations",                    # Solo para migraciones del sistema
    "oauth_clients",                 # Seguridad/autenticación interna
    "paynilo",                       # Integración de pagos externa
    "role_permissions",             # Control de permisos internos
    "template_versions",            # Versiones de plantillas, sin datos
    "templates",                    # Estructura visual, no analítica
    "user_data",                    # Solo datos de área/cargo del usuario
    "custom_fields",                # Campos personalizados (metadatos)
    "system_counters",              # Estadísticas internas del sistema
    "system_restrictions",          # Restricciones de uso/licencia
    "values_x_item",                # Datos técnicos por ítem
    "user_roles",
    "user_data",
    "user_headquarters",
    "term_and_conditions,",
    "severance_payments",
    "roles", 
    "role_permissions",
    "radian_events",
    "radian_documents",
    "plan_system_controller",
    "plan_electronic_documents",
    "api_access_tokens",
    "configurations",
    "company_areas",
    "consolidated_retention_certificates",
    "contact_accounts",
    "contact_relationship",
    "contact_statement",
    "contact",
    "custom_fields",
    "dining_tables",
    "ecomerce_legal_info",
    "ecomerce_purchase_orders",
    "electronic_payroll_test_set", 
    "electronic_contracts",
    "employee_positions", 
    "employees", 
    "migraciones ]"     # Registro técnico, no útil para KPIs
   
]


 

In [43]:
# ⚠️ Solo si tienes respaldo y estás en entorno seguro
# ELIMINAR TABLAS FINALES ¡OJO HAY QUE ESCOGER ANTES DE EJECUTAR!

# Desactivar restricciones !NO SE HACE ANTES DE VERIFICAR QUE NO GENERA NINGÚN DAÑO!
cursor.execute("SET FOREIGN_KEY_CHECKS=0;")

borrar_lista_tablas(tablas_a_eliminar)

# Volver a activar restricciones !NO SE HACE ANTES DE VERIFICAR QUE NO GENERA NINGÚN DAÑO!
cursor.execute("SET FOREIGN_KEY_CHECKS=1;")


❌ Error al eliminar api_access_tokens: 1051 (42S02): Unknown table 'erp.api_access_tokens'
❌ Error al eliminar electronic_payroll_test_set: 1051 (42S02): Unknown table 'erp.electronic_payroll_test_set'
❌ Error al eliminar migrations: 1051 (42S02): Unknown table 'erp.migrations'
❌ Error al eliminar oauth_clients: 1051 (42S02): Unknown table 'erp.oauth_clients'
❌ Error al eliminar paynilo: 1051 (42S02): Unknown table 'erp.paynilo'
❌ Error al eliminar role_permissions: 1051 (42S02): Unknown table 'erp.role_permissions'
✅ Tabla eliminada: template_versions
❌ Error al eliminar templates: 1051 (42S02): Unknown table 'erp.templates'
❌ Error al eliminar user_data: 1051 (42S02): Unknown table 'erp.user_data'
❌ Error al eliminar custom_fields: 1051 (42S02): Unknown table 'erp.custom_fields'


❌ Error al eliminar system_counters: 1051 (42S02): Unknown table 'erp.system_counters'
❌ Error al eliminar system_restrictions: 1051 (42S02): Unknown table 'erp.system_restrictions'
❌ Error al eliminar values_x_item: 1051 (42S02): Unknown table 'erp.values_x_item'
❌ Error al eliminar user_roles: 1051 (42S02): Unknown table 'erp.user_roles'
❌ Error al eliminar user_data: 1051 (42S02): Unknown table 'erp.user_data'
❌ Error al eliminar user_headquarters: 1051 (42S02): Unknown table 'erp.user_headquarters'
❌ Error al eliminar term_and_conditions,: 1051 (42S02): Unknown table 'erp.term_and_conditions,'
❌ Error al eliminar severance_payments: 1051 (42S02): Unknown table 'erp.severance_payments'
❌ Error al eliminar roles: 1051 (42S02): Unknown table 'erp.roles'
❌ Error al eliminar role_permissions: 1051 (42S02): Unknown table 'erp.role_permissions'
❌ Error al eliminar radian_events: 1051 (42S02): Unknown table 'erp.radian_events'
❌ Error al eliminar radian_documents: 1051 (42S02): Unknown tab

In [44]:
# Cerrar conexión
cursor.close()
conexion.close()
